Importing and File Location

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

#creating Paths
path= r'C:\Users\wjzak\Dropbox\Career Foundry\Data Immersion\Module 4- Python Fundamentals for Data Analysts\07-2025 Instacart Basket Analysis'

#using the path to pull file name and read orders.csv
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'orders.csv'), index_col = False)
#using the path to pull file name and read products.csv
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)

In [2]:
#print column names to use in aggregating below
df_ords.columns

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')

In [3]:
df_prods.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices'], dtype='object')

Data Consistency Exercise

In [2]:
df_ords.value_counts()

order_id  user_id  eval_set  order_number  order_dow  order_hour_of_day  days_since_prior_order
3421083   25247    prior     24            2          6                  21.0                      1
1         112108   train     4             4          10                 9.0                       1
2         202279   prior     3             5          9                  8.0                       1
3         205970   prior     16            5          17                 12.0                      1
4         178520   prior     36            1          9                  7.0                       1
                                                                                                  ..
15        54901    prior     51            3          11                 2.0                       1
14        18194    prior     49            3          15                 3.0                       1
13        45082    prior     2             6          17                 1.0                    

In [3]:
df_prods.value_counts()

product_id  product_name                                          aisle_id  department_id  prices
18458       Ranger IPA                                            27        5              9.2       2
462         Fiber 4g Gummy Dietary Supplement                     70        11             4.8       2
26808       Black House Coffee Roasty Stout Beer                  27        5              13.4      2
35306       Gluten Free Organic Peanut Butter & Chocolate Cereal  121       14             6.8       2
35491       Adore Forever Body Wash                               127       11             9.9       2
                                                                                                    ..
49684       Vodka, Triple Distilled, Twist of Vanilla             124       5              5.3       1
49685       En Croute Roast Hazelnut Cranberry                    42        1              3.1       1
49686       Artisan Baguette                                      112       3 

In [4]:
#data check- make sure the values and summary stats make sense for the variables
df_ords.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


2) Checking Orders Dataframe-
The data for order hour of the day looks a little off. You would expect this to be between 0 and 24 hours and the max is 2.3 with a mean of 1.3. I would take some time to review this a bit further to see how this variable is captured and represented here.

In [5]:
#testing is the the columns have mixed values
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].map(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)
#if no result printed, none of the data is mixed.
#nothing was printed and thus all data here is not mixed

In [6]:
#checking for null values
df_ords.isnull().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

4) Null Value Findings-
There is a signficiant amount of missing data found in the days since prior order category. This is likely the result of some of the users not making a second order. They are single instance users and their information could be important to understand.
I would not remove this data but instead create a subset to compare between multi-instance users and the this individual user. The best method here would be to create a new variable that flags based on the missing value. This would create the two subsets of the user population

In [7]:
#creates a variable for users that have only completed a single order
#This variable is coded 1 for being a new user and 0 for being a repeated user
df_ords['new_user']= df_ords['days_since_prior_order'].isna().astype(int)
df_ords.shape

(3421083, 8)

In [8]:
#look for full duplicates
df_ords_dups = df_ords[df_ords.duplicated()]
df_ords_dups.shape
#there are no duplciates founds in this data based on the reported info below.

(0, 8)

In [9]:
#Exporting the cleaned Data for Orders
df_ords.to_csv(os.path.join(path, '02 Data','Prepared Data', 'orders_clean.csv'))

**All info below is from the exercise or relates to the Products cleaning

In [10]:
#create test dataframe
df_test = pd.DataFrame()
#create a mixed type column
df_test['mix'] = ['a','b', 1, True]

df_test.head()

,mix
0,a
1,b
2,1
3,True


In [11]:
#Function for testing if a dataframe is mixed-type column- this a forloop that checks and
# returns if there is a value that is problematic. It produced the term mix when it finds this.
for col in df_test.columns.tolist():
  weird = (df_test[[col]].map(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

mix


In [12]:
#This will convert the data type from numeric to string
df_test['mix'] = df_test['mix'].astype('str')

In [13]:
#Finding Missing Values
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [14]:
#create subset of the 16 missing values
df_nan = df_prods[df_prods['product_name'].isnull() == True]
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [15]:
#REPLACE MISSING VALUES CODE IN #
#replace missing values with the mean
#df['column with missings'].fillna(mean value, inplace=True)
#replace missing with median
#df['column with missings'].fillna(median value, inplace=True)

#shape of the data
df_prods.shape

(49693, 5)

In [16]:
#creata a new data from without missing data
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]
df_prods_clean.shape

(49677, 5)

In [17]:
#Command to drop all missing values
df_prods.dropna(inplace = True)

#Command to drop NaNs from a specific column
df_prods.dropna(subset = ['product_name'], inplace = True)

In [18]:
#look for full duplicates
df_dups = df_prods_clean[df_prods_clean.duplicated()]
df_dups.shape

(5, 5)

In [19]:
#drop duplicates
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()
df_prods_clean_no_dups.shape

(49672, 5)

In [38]:
#export data after clean and no dupes
df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data','Prepared Data', 'products_clean_no_dupe.csv'))